In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import cycle
import datetime as dt
from torch.autograd import Variable
import random 
import os
from matplotlib.pyplot import figure
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import time 
import torch.nn.functional as F
import warnings
from fastprogress.fastprogress import master_bar, progress_bar
import torch.nn as nn
import torch.optim as optim
from copy import deepcopy
%matplotlib inline

warnings.filterwarnings("ignore")
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

 

## Set device parameters

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device is:",device)

device is: cuda


In [3]:
fill_type = "mean"
seq_length = 120
labels_length = 30

In [4]:
df_train = pd.read_csv(f"advance_work/{fill_type}/train.csv")
df_val = pd.read_csv(f"advance_work/{fill_type}/val.csv")

In [5]:
def meets_threshold(value, threshold=1e-5):
    return value >= threshold

# Function to build the dataset
def build_dataset(df):
    X, Y = [], []
    xl_col = df['xl'].values  
    for i in range(len(xl_col) - (seq_length + labels_length)):
        if meets_threshold(xl_col[i + seq_length]):
            x_ = xl_col[i:i + seq_length].tolist()
            x_ = [[-np.log(v)] for v in x_]
            X.append(np.array(x_))
            y_ = xl_col[i + seq_length:i + seq_length + labels_length].tolist()
            y_ = [[-np.log(v)] for v in y_]
            Y.append(np.array(y_))
    
    return X, Y

In [6]:
train_X, train_y = build_dataset(df_train)
valid_X, valid_y = build_dataset(df_val)

In [7]:
import gc
del df_test, df_train, df_val
gc.collect()

4

In [8]:

trainX = Variable(torch.Tensor(train_X))
trainy = Variable(torch.Tensor(train_y))

validX = Variable(torch.Tensor(valid_X))
validy= Variable(torch.Tensor(valid_y))

print ("trainX shape is:",trainX.size())
print ("trainy shape is:",trainy.size())
print ("validX shape is:",validX.size())
print ("validy shape is:",validy.size())

trainX shape is: torch.Size([191767, 120, 1])
trainy shape is: torch.Size([191767, 30, 1])
validX shape is: torch.Size([31875, 120, 1])
validy shape is: torch.Size([31875, 30, 1])


### Summery of the dataset used for all the experiments
```
trainX shape is: torch.Size([58024, 120, 1])
trainy shape is: torch.Size([58024, 30, 1])
validX shape is: torch.Size([10802, 120, 1])
validy shape is: torch.Size([10802, 30, 1])
```
```
min method
trainX shape is: torch.Size([191767, 120, 1])
trainy shape is: torch.Size([191767, 30, 1])
validX shape is: torch.Size([31875, 120, 1])
validy shape is: torch.Size([31875, 30, 1])
```

```
mean method
trainX shape is: torch.Size([191767, 120, 1])
trainy shape is: torch.Size([191767, 30, 1])
validX shape is: torch.Size([31875, 120, 1])
validy shape is: torch.Size([31875, 30, 1])
```

```
max method
trainX shape is: torch.Size([392794, 120, 1])
trainy shape is: torch.Size([392794, 30, 1])
validX shape is: torch.Size([253918, 120, 1])
validy shape is: torch.Size([253918, 30, 1])
```
-------------------------------------------------
Test set
```
TestX shape is: torch.Size([13954, 120, 1])
Testy shape is: torch.Size([13954, 30, 1])
```

# Building the Seq2Seq Model <a id="5"></a>

In [9]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class Encoder(nn.Module):
    """
    Encoder for the Seq2Seq model
    """
    def __init__(self, seq_len, n_features, embedding_dim=32):
        super(Encoder, self).__init__()
        self.seq_len, self.n_features = seq_len, n_features
        self.embedding_dim, self.hidden_dim = embedding_dim, 16  # Set hidden_dim to 16
        self.num_layers = 1
        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=self.hidden_dim,
            num_layers=1,
            batch_first=True,
            dropout=0.35
        )
   
    def forward(self, x):
        device = x.device
        x = x.reshape((1, self.seq_len, self.n_features))
        h_1 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device))
        c_1 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device))
        x, (hidden, cell) = self.lstm(x, (h_1, c_1))
        return x, hidden, cell

class Decoder(nn.Module):
    """
    Decoder for the Seq2Seq model
    """
    def __init__(self, seq_len, input_dim=32, n_features=1):
        super(Decoder, self).__init__()
        self.seq_len, self.input_dim = seq_len, input_dim
        self.hidden_dim, self.n_features = 16, n_features 
        self.lstm = nn.LSTM(
            input_size=1,
            hidden_size=16,  
            num_layers=1,
            batch_first=True,
            dropout=0.35
        )
        self.output_layer = nn.Linear(self.hidden_dim, n_features)

    def forward(self, x, input_hidden, input_cell):
        device = x.device
        x = x.reshape((1, 1, 1))
        x, (hidden_n, cell_n) = self.lstm(x, (input_hidden, input_cell))
        x = self.output_layer(x)
        return x, hidden_n, cell_n

class Seq2Seq(nn.Module):
    """
    Seq2Seq model
    """
    def __init__(self, seq_len, n_features, embedding_dim=32, output_length=labels_length):
        super(Seq2Seq, self).__init__()
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.encoder = Encoder(seq_len, n_features, embedding_dim).to(device)
        self.output_length = output_length
        self.decoder = Decoder(seq_len, embedding_dim, n_features).to(device)

    def forward(self, x, prev_y):
        encoder_output, hidden, cell = self.encoder(x)
        targets_ta = []
        prev_output = prev_y
        for out_days in range(self.output_length):
            prev_x, prev_hidden, prev_cell = self.decoder(prev_output, hidden, cell)
            hidden, cell = prev_hidden, prev_cell
            prev_output = prev_x
            targets_ta.append(prev_x.reshape(1))
        targets = torch.stack(targets_ta)
        return targets


In [10]:
n_features = 1
model = Seq2Seq(seq_length, n_features)
model = model.to(device)
model

Seq2Seq(
  (encoder): Encoder(
    (lstm): LSTM(1, 16, batch_first=True, dropout=0.35)
  )
  (decoder): Decoder(
    (lstm): LSTM(1, 16, batch_first=True, dropout=0.35)
    (output_layer): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [11]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (lstm): LSTM(1, 16, batch_first=True, dropout=0.35)
  )
  (decoder): Decoder(
    (lstm): LSTM(1, 16, batch_first=True, dropout=0.35)
    (output_layer): Linear(in_features=16, out_features=1, bias=True)
  )
)

#### Hyper-parameters and other settings

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=4e-3,weight_decay=1e-5)
criterion = torch.nn.MSELoss().to(device) 
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 5e-3, eta_min=1e-8, last_epoch=-1)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,  patience=10, factor =0.5 ,min_lr=1e-7, eps=1e-08)

#### Training loop

In [13]:
def train_model(model, TrainX, Trainy, ValidX, Validy, seq_length, n_epochs):
    history = dict(train=[], val=[])
    best_loss = 10000.0
    epochs_no_improve = 0
    early_stop_threshold = 5  # Stop if no improvement in validation loss for 5 consecutive epochs
    best_model_wts = deepcopy(model.state_dict())  # Deep copy the model's state_dict

    mb = master_bar(range(1, n_epochs + 1))

    for epoch in mb:
        print(epoch, dt.datetime.now())
        model.train()  # Set model to training mode

        train_losses = []
        for i in progress_bar(range(TrainX.size()[0]), parent=mb):
            seq_inp = TrainX[i, :, :].to(device)
            seq_true = Trainy[i, :, :].to(device)

            optimizer.zero_grad()
            seq_pred = model(seq_inp.unsqueeze(0), seq_inp[seq_length-1:seq_length, :].unsqueeze(0))
            loss = criterion(seq_pred.squeeze(0), seq_true)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()

            train_losses.append(loss.item())

        # Validation phase
        val_losses = []
        model.eval()  # Set model to evaluation mode
        with torch.no_grad():
            for i in progress_bar(range(ValidX.size()[0]), parent=mb):
                seq_inp = ValidX[i, :, :].to(device)
                seq_true = Validy[i, :, :].to(device)

                seq_pred = model(seq_inp.unsqueeze(0), seq_inp[seq_length-1:seq_length, :].unsqueeze(0))
                loss = criterion(seq_pred.squeeze(0), seq_true)
                val_losses.append(loss.item())

        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)

        history['train'].append(train_loss)
        history['val'].append(val_loss)

        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = deepcopy(model.state_dict())
            epochs_no_improve = 0  # Reset counter
            torch.save(model.state_dict(), f"best_model_{fill_type}_seq_{seq_length}_label_{labels_length}.pt")
            print("Saved best model at epoch:", epoch, "with val loss:", val_loss)
        else:
            epochs_no_improve += 1

        print(f'Epoch {epoch}: train loss {train_loss} val loss {val_loss}')
        scheduler.step()

        if epochs_no_improve >= early_stop_threshold:
            print("Early stopping triggered")
            break

    model.load_state_dict(best_model_wts)  # Load the best model weights
    return model.eval(), history

In [14]:
model, history = train_model(
  model,
  trainX,trainy,
  validX,validy,
  seq_length,
  n_epochs=100, 
)

1 2024-02-14 03:23:06.167417
Saved best model at epoch: 1 with val loss: 2.2829366452566404
Epoch 1: train loss 0.6022317319466128 val loss 2.2829366452566404
2 2024-02-14 04:19:33.143370
Saved best model at epoch: 2 with val loss: 1.204550441638676
Epoch 2: train loss 0.5791567322898116 val loss 1.204550441638676
3 2024-02-14 05:10:25.538376
Epoch 3: train loss 0.5754673193211568 val loss 1.2572883906289807
4 2024-02-14 06:01:19.707102
Saved best model at epoch: 4 with val loss: 1.0530952712864268
Epoch 4: train loss 0.5752455430415572 val loss 1.0530952712864268
5 2024-02-14 06:52:11.696724
Epoch 5: train loss 0.5697756685795625 val loss 1.6135796598918224
6 2024-02-14 07:43:06.294384
Saved best model at epoch: 6 with val loss: 1.009132662061821
Epoch 6: train loss 0.5885446973673445 val loss 1.009132662061821
7 2024-02-14 08:34:01.355040
Epoch 7: train loss 0.5831711863411165 val loss 1.1534799064719647
8 2024-02-14 09:24:53.784449
Epoch 8: train loss 0.5849430398428584 val loss 1.2

## Test Results

In [20]:
## Prepare the test dataset
fill_type = "fractal"
df_test = pd.read_csv(f"advance_work/{fill_type}/test.csv")
test_X, test_y = build_dataset(df_test)
test_X = Variable(torch.Tensor(test_X))
test_y = Variable(torch.Tensor(test_y))

#### Test results with fractal imputation

In [21]:
fill_type = "fractal"
model.load_state_dict(torch.load(f"best_model_{fill_type}_seq_{seq_length}_label_{labels_length}.pt"))
model = model.eval()
test_losses = []
with torch.no_grad():
    for i in progress_bar(range(test_X.size()[0])):
        seq_inp = test_X[i,:,:].to(device)
        seq_true = test_y[i,:,:].to(device)

        seq_pred = model(seq_inp,seq_inp[seq_length-1:seq_length,:])
        

        loss = criterion(seq_pred, seq_true)
        test_losses.append(loss.item())

val_loss = np.mean(test_losses)
print(val_loss)

1.8678501524936588


#### Test results with min imputation

In [22]:
fill_type = "min"

model.load_state_dict(torch.load(f"best_model_{fill_type}_seq_{seq_length}_label_{labels_length}.pt"))
model = model.eval()
test_losses = []
with torch.no_grad():
    for i in progress_bar(range(test_X.size()[0])):
        seq_inp = test_X[i,:,:].to(device)
        seq_true = test_y[i,:,:].to(device)

        seq_pred = model(seq_inp,seq_inp[seq_length-1:seq_length,:])
        

        loss = criterion(seq_pred, seq_true)
        test_losses.append(loss.item())

val_loss = np.mean(test_losses)
print(val_loss)

0.9625936843656333


#### Test results with mean imputation

In [23]:
fill_type = "mean"

model.load_state_dict(torch.load(f"best_model_{fill_type}_seq_{seq_length}_label_{labels_length}.pt"))
model = model.eval()
test_losses = []
with torch.no_grad():
    for i in progress_bar(range(test_X.size()[0])):
        seq_inp = test_X[i,:,:].to(device)
        seq_true = test_y[i,:,:].to(device)

        seq_pred = model(seq_inp,seq_inp[seq_length-1:seq_length,:])
        

        loss = criterion(seq_pred, seq_true)
        test_losses.append(loss.item())

val_loss = np.mean(test_losses)
print(val_loss)

0.6606694676633204


#### Test results with max imputation

In [24]:
fill_type = "max"

model.load_state_dict(torch.load(f"best_model_{fill_type}_seq_{seq_length}_label_{labels_length}.pt"))
model = model.eval()
test_losses = []
with torch.no_grad():
    for i in progress_bar(range(test_X.size()[0])):
        seq_inp = test_X[i,:,:].to(device)
        seq_true = test_y[i,:,:].to(device)

        seq_pred = model(seq_inp,seq_inp[seq_length-1:seq_length,:])
        

        loss = criterion(seq_pred, seq_true)
        test_losses.append(loss.item())

val_loss = np.mean(test_losses)
print(val_loss)

30.367870760558585
